# Cuaderno Clase PLN - GloVe y FastText

**Objetivos:**
*   Conocer enfoques alternativos a Word2Vec: GloVe (conteos globales) y FastText (subpalabras).
*   Entender la ventaja clave de FastText para manejar palabras fuera de vocabulario (OOV - Out Of Vocabulary).
*   Cargar y usar vectores FastText pre-entrenados.
*   Comparar resultados y capacidades (especialmente OOV) con Word2Vec.
*   Reflexionar sobre cómo evaluar la calidad de los embeddings y detectar sesgos.

**Agenda:**
1.  Instalaciones e Importaciones
2.  Repaso Rápido: Word2Vec y sus limitaciones (OOV)
3.  GloVe: Vectores Globales desde Co-ocurrencias
4.  FastText: El Poder de las Subpalabras (¡Adiós OOV!)
5.  Cargando Vectores FastText Pre-entrenados
6.  Explorando FastText: Similitud, Analogías y ¡OOV!
7.  Comparativa: Word2Vec vs FastText (foco en OOV)
8.  Micro-Laboratorio (Ejercicio Práctico)
9.  Brainstorming: Evaluación y Detección de Sesgos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Instalaciones e Importaciones

In [2]:
# Necesitamos gensim principalmente
!pip install gensim > /dev/null

In [3]:
!pip uninstall gensim -y # Remove the existing gensim installation
!pip install gensim # Reinstall gensim to align with the NumPy version
!pip install scipy==1.10.1
# Restart the kernel to ensure the changes take effect

Found existing installation: gensim 4.3.3
Uninstalling gensim-4.3.3:
  Successfully uninstalled gensim-4.3.3
  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)


In [4]:
import gensim
from gensim.models import KeyedVectors, FastText # Ahora importamos FastText también
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Librerías importadas.")

# **Importante:** Para este cuaderno, idealmente necesitaremos:
# 1. El modelo Word2Vec cargado previamente (para comparar).
# 2. Un modelo FastText pre-entrenado para español (¡necesitamos descargarlo!).

Librerías importadas.


# 2. Repaso Rápido: Word2Vec y sus limitaciones (OOV)

Recordemos:
*   Word2Vec (CBOW/Skip-gram) aprende embeddings prediciendo palabras en contextos locales.
*   Genera vectores densos que capturan semántica (similitud, analogías).
*   Usamos modelos pre-entrenados porque entrenarlos es costoso.

**Una limitación importante:** Word2Vec asigna un vector a cada palabra *completa* que vio durante el entrenamiento. Si en tiempo de uso aparece una palabra que **NO estaba en el vocabulario original** (Out-Of-Vocabulary - OOV):
*   Un error tipográfico ("computadorra").
*   Una palabra nueva o muy rara ("covid", "guasap").
*   Una variante morfológica no vista ("cantábamos").

**¿Qué pasa con Word2Vec?** ¡Generalmente da un error (`KeyError`) o asigna un vector genérico de "desconocido" (UNK - Unknown), perdiendo toda información específica! Esto es un problema en aplicaciones reales.

# 3. GloVe: Vectores Globales desde Co-ocurrencias

GloVe (Global Vectors for Word Representation) de Stanford es otra técnica popular para obtener embeddings.

*   **Enfoque Diferente:** En lugar de predecir contextos locales (como Word2Vec), GloVe se enfoca en las **estadísticas globales de co-ocurrencia** de palabras en todo el corpus.
*   **Idea Central:** La *relación* entre las probabilidades de co-ocurrencia de dos palabras con una tercera palabra "sonda" puede revelar información sobre su significado. Por ejemplo, la probabilidad de que "hielo" aparezca cerca de "sólido" será alta, mientras que cerca de "gas" será baja. La probabilidad de que "vapor" aparezca cerca de "gas" será alta y cerca de "sólido" baja. GloVe modela estas *proporciones* de probabilidades.
*   **Entrenamiento:** Construye una matriz gigante de co-ocurrencia (cuántas veces la palabra X aparece cerca de la palabra Y) y luego usa factorización de matrices para encontrar los vectores de palabras que mejor explican esa matriz.
*   **Resultado:** Embeddings densos similares a Word2Vec, a menudo muy buenos para tareas de similitud y analogía.
*   **Limitación OOV:** Al igual que Word2Vec, GloVe tradicionalmente opera a nivel de palabra completa, por lo que también sufre del problema de OOV.

*(Nota: Encontrar modelos GloVe pre-entrenados y de buena calidad para español puede ser más difícil que para Word2Vec o FastText. A menudo se distribuyen en formato texto).*

# 4. FastText: El Poder de las Subpalabras (¡Adiós OOV!)

FastText, desarrollado por Facebook AI Research (FAIR), es una extensión inteligente de Word2Vec (Skip-gram) que aborda directamente el problema OOV.

*   **¡La Gran Idea!:** FastText no trata las palabras como unidades indivisibles. Representa cada palabra como una **bolsa de n-gramas de caracteres**, además de la palabra completa.
    *   Ejemplo: Para la palabra "amarillo" y usando n=3 (trigramas):
        *   Se consideran n-gramas como: `<am`, `ama`, `mar`, `ari`, `ril`, `ill`, `llo`, `lo>` (con `<` y `>` marcando inicio y fin).
        *   También se considera la palabra completa: `<amarillo>`
*   **Aprendizaje:** El modelo aprende vectores no solo para las palabras completas, sino **para todos los n-gramas de caracteres** vistos en el corpus.
*   **Vector Final:** El vector de una palabra se obtiene **sumando los vectores de todos sus n-gramas de caracteres** (y el vector de la palabra completa si existe).

**¡La Ventaja Clave - Manejo de OOV!**
*   Si aparece una palabra nueva que no estaba en el vocabulario (ej: "amarillento"), FastText **aún puede construir un vector para ella**. ¿Cómo? Sumando los vectores de los n-gramas que sí conoce y que forman parte de la palabra nueva (ej: `<am`, `ama`, `mar`, `ari`, `ril`, `ill`, `lle`, `len`, `ent`, `nto`, `to>`).
*   El vector resultante captura información de las "partes" de la palabra, lo que a menudo resulta en una representación razonable incluso para palabras desconocidas.

**Otros Beneficios:**
*   Funciona muy bien para **lenguajes morfológicamente ricos** (como el español) donde muchas palabras comparten raíces y afijos (ej: "nación", "nacional", "nacionalidad"). Comparten n-gramas, sus vectores estarán relacionados.
*   Es más robusto a **errores tipográficos** ("computadorra" compartirá muchos n-gramas con "computadora").

# 5. Cargando Vectores FastText Pre-entrenados

Al igual que con Word2Vec, lo usual es usar modelos FastText pre-entrenados.

**Tarea:** Descargar los vectores pre-entrenados para **español** desde el sitio oficial de FastText: [https://fasttext.cc/docs/en/pretrained-vectors.html](https://fasttext.cc/docs/en/pretrained-vectors.html)

*   **Importante:** Descargar el archivo `.bin`. Este formato contiene no solo los vectores de las palabras del vocabulario, sino también la información de los n-gramas necesaria para calcular vectores de palabras OOV. El formato `.vec` es más pequeño pero pierde esta capacidad OOV.
*   El archivo `.bin` para español también es grande (varios GB). Anoten la ruta donde lo guardan.

**¡Reemplacen `'RUTA_AL_ARCHIVO_FASTTEXT.bin'` con la ruta real!**

In [5]:
# --- ¡¡¡ MODIFICAR ESTA RUTA !!! ---
# Ejemplo de ruta (si lo subieron a Colab o está en el mismo directorio):
# path_to_fasttext = 'cc.es.300.bin' # Nombre común del archivo FastText para español
# Ejemplo de ruta (si está en Google Drive montado):
path_to_fasttext = '/content/drive/MyDrive/wiki.es.bin' # Ajustar según su estructura

In [6]:
# Intentar cargar el modelo FastText
try:
    print("Cargando vectores FastText (.bin)... (¡Esto puede tardar MUCHO tiempo y consumir RAM!)")
    # Usamos FastText.load_fasttext_format para cargar modelos .bin de FastText
    fasttext_model = gensim.models.fasttext.load_facebook_model(path_to_fasttext)
    # Los vectores están dentro del atributo .wv (como en Word2Vec cargado con KeyedVectors)
    fasttext_vectors = fasttext_model.wv
    print(f"¡Vectores FastText cargados! Vocabulario (estimado): {len(fasttext_vectors.index_to_key)} palabras. Dimensión: {fasttext_vectors.vector_size}")
    # Nota: El tamaño del vocabulario explícito puede ser menor en .bin, pero puede generar para OOV.
except FileNotFoundError:
    print(f"Error: No se encontró el archivo FastText en la ruta '{path_to_fasttext}'.")
    print("Por favor, descarga el archivo .bin pre-entrenado para español desde el sitio de FastText")
    print("y asegúrate de que la variable 'path_to_fasttext' tenga la ruta correcta.")
    fasttext_vectors = None
except Exception as e:
    print(f"Ocurrió un error al cargar el modelo FastText: {e}")
    fasttext_vectors = None

Cargando vectores FastText (.bin)... (¡Esto puede tardar MUCHO tiempo y consumir RAM!)
¡Vectores FastText cargados! Vocabulario (estimado): 985667 palabras. Dimensión: 300


In [7]:
# También intentemos recargar el modelo Word2Vec del martes para comparar
# --- ¡¡¡ MODIFICAR ESTA RUTA TAMBIÉN SI ES NECESARIO !!! ---
path_to_word2vec = '/content/drive/MyDrive/SBW-vectors-300-min5.bin.gz' # La ruta del martes
word2vec_vectors = None

In [8]:
try:
    print("\nRecargando vectores Word2Vec del martes...")
    word2vec_vectors = KeyedVectors.load_word2vec_format(path_to_word2vec, binary=True)
    print("Vectores Word2Vec recargados.")
except Exception as e:
    print(f"No se pudo recargar Word2Vec desde '{path_to_word2vec}': {e}")


Recargando vectores Word2Vec del martes...
Vectores Word2Vec recargados.


# 6. Explorando FastText: Similitud, Analogías y ¡OOV!

Si `fasttext_vectors` se cargó, podemos hacer pruebas similares a las de Word2Vec, pero prestando especial atención a las palabras OOV.

In [9]:
if fasttext_vectors:
    # --- Pruebas estándar (similitud, analogía) ---
    print("\n--- Explorando FastText ---")
    # Similitud
    try:
        similares_rey_ft = fasttext_vectors.most_similar('rey', topn=5)
        print("FastText - Más similares a 'rey':", similares_rey_ft)
    except KeyError:
        print("FastText - Palabra 'rey' no encontrada.") # Raro si el modelo es bueno

    # Analogía
    try:
        analogia_reina_ft = fasttext_vectors.most_similar(positive=['rey', 'mujer'], negative=['hombre'], topn=1)
        print("FastText - rey - hombre + mujer ≈", analogia_reina_ft)
    except KeyError as e:
        print(f"FastText - Error en analogía rey/reina: falta la palabra '{e.args[0]}'")


    # --- ¡La prueba OOV! ---
    print("\n--- Probando Palabras Fuera de Vocabulario (OOV) ---")
    palabras_oov = [
        "computadorra", # Error tipográfico
        "wasapear",     # Neologismo / Palabra coloquial
        "covid",        # Palabra relativamente nueva (depende del corpus)
        "programadorazo", # Palabra inventada con sufijo común
        "desayunábamos", # Forma verbal menos frecuente
        "internauta"    # Palabra estándar, veamos si está
    ]

    for palabra in palabras_oov:
        print(f"\nPalabra OOV: '{palabra}'")
        # ¿Está explícitamente en el vocabulario? (Puede que sí si el corpus era reciente)
        in_vocab = palabra in fasttext_vectors
        print(f"  ¿En vocabulario explícito? {in_vocab}")

        # Intentar obtener el vector (¡FastText debería poder!)
        try:
            vector_oov = fasttext_vectors[palabra]
            print(f"  ¡Vector obtenido! (Dim: {vector_oov.shape})")

            # Encontrar similares basados en el vector generado
            similares_oov = fasttext_vectors.most_similar(palabra, topn=3)
            print(f"  Similares (FastText): {similares_oov}")
        except Exception as e:
            # Este error no debería ocurrir con un modelo .bin cargado correctamente
            print(f"  Error inesperado obteniendo vector/similares para '{palabra}': {e}")

else:
    print("\nNo se pudieron cargar los vectores FastText. Saltando la exploración.")


--- Explorando FastText ---
FastText - Más similares a 'rey': [('monarca', 0.7459232807159424), ('rey―', 0.6321003437042236), ('exmonarca', 0.6310685873031616), ('rey,', 0.6303314566612244), ('reina', 0.6260297298431396)]
FastText - rey - hombre + mujer ≈ [('reina', 0.6612293720245361)]

--- Probando Palabras Fuera de Vocabulario (OOV) ---

Palabra OOV: 'computadorra'
  ¿En vocabulario explícito? True
  ¡Vector obtenido! (Dim: (300,))
  Similares (FastText): [('computador', 0.8841244578361511), ('computadora/ordenador', 0.8638773560523987), ('computadora,', 0.8576205372810364)]

Palabra OOV: 'wasapear'
  ¿En vocabulario explícito? True
  ¡Vector obtenido! (Dim: (300,))
  Similares (FastText): [('dissapear', 0.6578547358512878), ('tapear', 0.6350954174995422), ('papear', 0.5647345781326294)]

Palabra OOV: 'covid'
  ¿En vocabulario explícito? True
  ¡Vector obtenido! (Dim: (300,))
  Similares (FastText): [('covides', 0.7112458944320679), ('coview', 0.6201539635658264), ('covida', 0.5898

# 7. Comparativa: Word2Vec vs FastText (foco en OOV)

Ahora, si ambos modelos (`word2vec_vectors` y `fasttext_vectors`) están cargados, podemos comparar directamente su comportamiento con palabras OOV.

In [10]:
# --- Comparación directa OOV ---
if word2vec_vectors and fasttext_vectors:
    print("\n--- Comparación OOV: Word2Vec vs FastText ---")

    # Reutilizamos la lista de palabras OOV
    palabras_oov = [
        "computadorra", "wasapear", "covid", "programadorazo", "desayunábamos", "internauta"
    ]

    for palabra in palabras_oov:
        print(f"\n--- Palabra: '{palabra}' ---")

        # Intentar con Word2Vec
        print("  Intentando con Word2Vec:")
        try:
            vector_w2v = word2vec_vectors[palabra]
            similares_w2v = word2vec_vectors.most_similar(palabra, topn=3)
            print(f"    ¡Encontrada! Vector: {vector_w2v.shape}, Similares: {similares_w2v}")
        except KeyError:
            print("    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).")
        except Exception as e:
            print(f"    ERROR inesperado con Word2Vec: {e}")

        # Intentar con FastText
        print("  Intentando con FastText:")
        try:
            vector_ft = fasttext_vectors[palabra]
            similares_ft = fasttext_vectors.most_similar(palabra, topn=3)
            print(f"    ¡Vector generado! Vector: {vector_ft.shape}, Similares: {similares_ft}")
        except Exception as e:
            # No debería fallar por KeyError, pero podría haber otro error
            print(f"    ERROR inesperado con FastText: {e}")

else:
    print("\nNo se pudieron cargar ambos modelos (Word2Vec y FastText) para comparar.")


--- Comparación OOV: Word2Vec vs FastText ---

--- Palabra: 'computadorra' ---
  Intentando con Word2Vec:
    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('computador', 0.8841244578361511), ('computadora/ordenador', 0.8638773560523987), ('computadora,', 0.8576205372810364)]

--- Palabra: 'wasapear' ---
  Intentando con Word2Vec:
    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('dissapear', 0.6578547358512878), ('tapear', 0.6350954174995422), ('papear', 0.5647345781326294)]

--- Palabra: 'covid' ---
  Intentando con Word2Vec:
    ERROR: Palabra no encontrada en el vocabulario Word2Vec (KeyError).
  Intentando con FastText:
    ¡Vector generado! Vector: (300,), Similares: [('covides', 0.7112458944320679), ('coview', 0.6201539635658264), ('covida', 0.5898890495300293)]

--- Palabra: 'progr

**Conclusión de la Comparativa:**

Deberían observar que para la mayoría (o todas) las palabras OOV, Word2Vec lanza un `KeyError`, mientras que FastText logra generar un vector y encontrar palabras similares (que pueden o no ser muy coherentes, dependiendo de qué tan "rara" sea la palabra OOV, pero *intenta* algo basado en sus partes).

**¿Cuándo usar FastText?**
*   Cuando trabajas con texto ruidoso (redes sociales, OCR) con typos.
*   Cuando tratas con lenguajes morfológicamente ricos (español, alemán, etc.).
*   Cuando esperas encontrar palabras nuevas o raras en tus datos de aplicación.
*   **En general, para español, FastText suele ser una opción más robusta y recomendable que Word2Vec o GloVe.**

**Desventaja:** Los modelos `.bin` de FastText son más grandes en disco y consumen más RAM porque almacenan información de los n-gramas.

# 8. Micro-Laboratorio (Ejercicio Práctico)

**Consigna:** (Asumiendo que `fasttext_vectors` y `word2vec_vectors` están cargados)

1.  **Comparación de Resultados:**
    *   Elegir 3 palabras que **sí** estén en ambos vocabularios (ej: 'gato', 'correr', 'inteligencia').
    *   Para cada palabra, obtener las 5 más similares usando `word2vec_vectors.most_similar()` y `fasttext_vectors.most_similar()`.
    *   Comparar las listas de similares. ¿Son idénticas? ¿Muy parecidas? ¿Diferentes? ¿Cuál les parece "mejor" o más coherente? Anotar observaciones.

2.  **Test OOV Exhaustivo:**
    *   Crear una lista propia de 10 palabras OOV. Incluyan:
        *   Errores tipográficos comunes (ej: "hobmre", "qeu", "dicimbre").
        *   Diminutivos/Aumentativos (ej: "perrito", "casita", "libraco").
        *   Formas verbales conjugadas (ej: "habíamos comido", "cantasteis").
        *   Palabras inventadas pero plausibles (ej: "tecnoestrés", "computofilia").
    *   Para **cada** palabra OOV de su lista:
        *   Verificar si da `KeyError` en `word2vec_vectors`.
        *   Obtener las 3 palabras más similares usando `fasttext_vectors`. Anotar los resultados. ¿Los similares que da FastText tienen algún sentido basado en las partes de la palabra OOV?

3.  **Discusión:**
    *   ¿En qué tipo de aplicación real (ej: un chatbot de atención al cliente, un sistema de recomendación de noticias, un corrector ortográfico) creen que la capacidad OOV de FastText marcaría una diferencia significativa respecto a usar Word2Vec? ¿Por qué?

### 1. Comparación de Resultados para Palabras en Vocabulario
Vamos a elegir tres palabras comunes que esperamos que estén en ambos vocabularios y comparar sus similares. Usaré ejemplos que probablemente estén en modelos grandes en inglés, ya que son más accesibles.

In [12]:
if fasttext_vectors and word2vec_vectors:
    print("\n--- Comparación de Similares para Palabras en Vocabulario ---")
    palabras_en_vocab = ['perro', 'volar', 'artificial'] # Elegidas para estar en modelos comunes en inglés

    for palabra in palabras_en_vocab:
        print(f"\nPalabra: '{palabra}'")

        # Similares con Word2Vec
        print("  Similares (Word2Vec):")
        try:
            similares_w2v = word2vec_vectors.most_similar(palabra, topn=5)
            for sim_word, score in similares_w2v:
                print(f"    - {sim_word}: {score:.4f}")
        except KeyError:
            print("    Palabra no encontrada en Word2Vec (esto no debería pasar para palabras comunes).")

        # Similares con FastText
        print("  Similares (FastText):")
        try:
            similares_ft = fasttext_vectors.most_similar(palabra, topn=5)
            for sim_word, score in similares_ft:
                print(f"    - {sim_word}: {score:.4f}")
        except KeyError:
            print("    Palabra no encontrada en FastText (esto no debería pasar para palabras comunes).")
else:
    print("\nNo se pudieron cargar ambos modelos para la comparación en vocabulario.")


--- Comparación de Similares para Palabras en Vocabulario ---

Palabra: 'perro'
  Similares (Word2Vec):
    - gato: 0.8182
    - cachorro: 0.7781
    - oso: 0.7136
    - perrito: 0.7128
    - rottweiler: 0.7068
  Similares (FastText):
    - perros: 0.7588
    - terrier: 0.6690
    - perrito: 0.6587
    - cachorro: 0.6577
    - gato: 0.6526

Palabra: 'volar'
  Similares (Word2Vec):
    - despegar: 0.7221
    - aterrizar: 0.7009
    - saltar: 0.6959
    - volando: 0.6743
    - vuela: 0.6551
  Similares (FastText):
    - volarse: 0.7906
    - volar,: 0.7677
    - volarlo: 0.7537
    - volarlos: 0.7331
    - volarle: 0.7325

Palabra: 'artificial'
  Similares (Word2Vec):
    - Inseminación: 0.5876
    - articial: 0.5663
    - artifical: 0.5657
    - Lago_Apanás: 0.5383
    - multijuegos: 0.5181
  Similares (FastText):
    - bioartificial: 0.7892
    - artificial»: 0.7790
    - artificial—: 0.7639
    - artificialmente: 0.7551
    - artificials: 0.7375


--- Observaciones de la Comparación en Vocabulario ---

¿Son las listas idénticas? Generalmente no.

¿Muy parecidas? A menudo sí, con solapamiento significativo de las palabras principales.

¿Diferentes? Pueden diferir en las palabras con menor puntuación o en matices sutiles.

¿Cuál les parece 'mejor' o más coherente? Esto es subjetivo y depende del caso de uso.
- **Word2Vec** tiende a ser muy bueno capturando relaciones semánticas precisas para palabras bien representadas en su corpus.
- **FastText**, al considerar subpalabras, a veces puede capturar mejor las relaciones morfológicas y puede tener un 'sentido' más robusto incluso para palabras menos frecuentes, ya que la información de subpalabras contribuye a su vector.
En general, para palabras comunes y bien representadas, ambos modelos suelen dar resultados muy coherentes y de alta calidad.

### 2. Test OOV Exhaustivo
Ahora, realizaremos el test OOV exhaustivo con una lista propia de 10 palabras OOV.

In [13]:
if fasttext_vectors and word2vec_vectors: # Aseguramos que ambos modelos estén cargados
    print("\n--- Test OOV Exhaustivo ---")

    # Lista propia de 10 palabras OOV (ejemplos en español, asumiendo un modelo en español idealmente)
    # Si los modelos cargados son en inglés, los resultados para estas palabras en español
    # serán de 'KeyError' para Word2Vec y vectores menos significativos para FastText.
    # Para una demostración real en español, necesitarías modelos pre-entrenados en español.
    palabras_oov_personalizadas = [
        "hobmre",          # Error tipográfico
        "qeu",             # Error tipográfico común
        "dicimbre",        # Error tipográfico
        "perrito",         # Diminutivo
        "casitass",        # Diminutivo/Aumentativo con error
        "libraco",         # Aumentativo
        "habíamoscomido",  # Forma verbal conjugada (sin espacio)
        "cantasteis",      # Forma verbal menos frecuente
        "tecnoestrés",     # Palabra inventada (tecnología + estrés)
        "computofilia",    # Palabra inventada (computadora + filia)
    ]

    for palabra in palabras_oov_personalizadas:
        print(f"\n--- Palabra OOV: '{palabra}' ---")

        # Verificar con Word2Vec
        print("  Verificando con Word2Vec:")
        try:
            vector_w2v = word2vec_vectors[palabra]
            print(f"    ¡Word2Vec encontró la palabra! (Esto es inesperado para una OOV real): {vector_w2v.shape}")
        except KeyError:
            print("    **Word2Vec: KeyError.** Palabra no encontrada en su vocabulario. (¡Comportamiento esperado para OOV!)")
        except Exception as e:
            print(f"    Error inesperado con Word2Vec: {e}")

        # Obtener similares con FastText
        print("  Obteniendo similares con FastText:")
        try:
            vector_ft = fasttext_vectors[palabra]
            print(f"    FastText: Vector obtenido. (Dim: {vector_ft.shape})")
            similares_ft = fasttext_vectors.most_similar(palabra, topn=3)
            print(f"    Similares (FastText): {similares_ft}")

            # Análisis de sentido para FastText
            print("    Análisis de sentido de similares (FastText):")
            for sim_palabra, score in similares_ft:
                # Aquí la observación es manual: ¿La palabra similar tiene sentido con los componentes de la OOV?
                # Ej: para "computadorra", ¿da "computadora", "ordenador"?
                # Para "perrito", ¿da "perro", "cachorro"?
                # Para "tecnoestrés", ¿da "tecnología", "estrés"?
                print(f"      - '{sim_palabra}': {score:.4f}")
            print("    ¿Los similares de FastText tienen sentido basado en las partes de la palabra OOV? (Observación manual)")

        except Exception as e:
            print(f"    Error inesperado con FastText: {e}")
            print("    (Este error es raro si el modelo FastText está cargado correctamente, ya que está diseñado para manejar OOV).")

else:
    print("\nAmbos modelos (Word2Vec y FastText) no están cargados para el test OOV exhaustivo.")


--- Test OOV Exhaustivo ---

--- Palabra OOV: 'hobmre' ---
  Verificando con Word2Vec:
    **Word2Vec: KeyError.** Palabra no encontrada en su vocabulario. (¡Comportamiento esperado para OOV!)
  Obteniendo similares con FastText:
    FastText: Vector obtenido. (Dim: (300,))
    Similares (FastText): [('nobmre', 0.6312950253486633), ('ademre', 0.5202481746673584), ('demre', 0.5149299502372742)]
    Análisis de sentido de similares (FastText):
      - 'nobmre': 0.6313
      - 'ademre': 0.5202
      - 'demre': 0.5149
    ¿Los similares de FastText tienen sentido basado en las partes de la palabra OOV? (Observación manual)

--- Palabra OOV: 'qeu' ---
  Verificando con Word2Vec:
    ¡Word2Vec encontró la palabra! (Esto es inesperado para una OOV real): (300,)
  Obteniendo similares con FastText:
    FastText: Vector obtenido. (Dim: (300,))
    Similares (FastText): [('qeuab', 0.6885186433792114), ('qeuad', 0.6670765280723572), ('wikisilki/pi', 0.6110560297966003)]
    Análisis de sentido d

### 3. Discusión: Impacto de la Capacidad OOV de FastText
La capacidad de FastText para generar vectores para palabras fuera de su vocabulario (OOV) es una ventaja significativa sobre modelos como Word2Vec que operan a nivel de palabra completa y simplemente fallan (KeyError) cuando encuentran un término desconocido.

¿En qué tipo de aplicaciones reales la capacidad OOV de FastText marcaría una diferencia significativa?

Chatbots y Asistentes Virtuales de Atención al Cliente:

¿Por qué? Los usuarios a menudo cometen errores tipográficos ("Nesecito ayua"), usan neologismos ("whatsappear", "googlear"), lenguaje coloquial o abreviaciones que no se vieron durante el entrenamiento. Word2Vec simplemente no entendería estas palabras, lo que llevaría a respuestas irrelevantes o a la incapacidad de procesar la consulta. FastText, al analizar subpalabras, puede inferir el significado de "nesecito" a partir de "necesito" o de "wasapear" a partir de "whatsapp", mejorando drásticamente la comprensión de la intención del usuario y la calidad del servicio.
Sistemas de Recomendación de Noticias o Contenido:

¿Por qué? Las noticias están en constante evolución, introduciendo nuevos nombres de personas, lugares, eventos ("el presidente X", "la ciudad de Y", "la nueva variante Z"). Un modelo Word2Vec se quedaría ciego ante estas novedades. FastText podría, por ejemplo, entender que "covid-19" (si no estaba en su vocabulario original) está relacionado con "virus" o "pandemia" debido a los n-gramas de caracteres compartidos, permitiendo que el sistema siga recomendando contenido relevante sobre temas emergentes.
Correctores Ortográficos y Sugerencia de Texto:

¿Por qué? Esta es una aplicación directa. Un corrector ortográfico basado en Word2Vec no sabría qué hacer con una palabra mal escrita. FastText, sin embargo, puede generar un vector para la palabra incorrecta y luego encontrar palabras similares en su vocabulario que tengan muchos n-gramas en común, lo que le permite sugerir la corrección correcta (ej., "computadorra" -> "computadora"). Esto mejora la precisión y la utilidad de estas herramientas.
Análisis de Sentimiento en Redes Sociales o Plataformas de Opinión:

¿Por qué? El lenguaje en redes sociales es muy dinámico, informal y propenso a errores tipográficos, abreviaturas y neologismos. Un modelo Word2Vec lucharía para capturar el sentimiento de frases que contienen muchas palabras OOV. FastText puede inferir el significado y, por ende, el sentimiento de esas palabras desconocidas, llevando a un análisis de sentimiento más robusto y preciso en contextos informales.

# 9. Brainstorming: Evaluación y Detección de Sesgos

Hemos visto diferentes formas de crear embeddings (Word2Vec, GloVe, FastText) y cómo usarlos. Pero, ¿cómo sabemos si un conjunto de embeddings es "bueno"? ¿Y cómo abordamos el problema de los sesgos que mencionamos en clases anteriores?

**Pregunta:** **¿Cómo podemos evaluar la calidad de los word embeddings y detectar posibles sesgos?**

*   **Evaluación Intrínseca:**
    *   Medir qué tan bien funcionan los embeddings en tareas específicas relacionadas con las palabras mismas, **sin** una aplicación final.
    *   Ejemplos:
        *   **Similitud de Palabras:** Comparar la similitud coseno entre pares de palabras según los embeddings, con juicios de similitud dados por humanos (datasets estándar como WordSim-353, SimLex-999, adaptados o creados para español).
        *   **Tareas de Analogía:** Ver qué tan bien resuelven analogías (`rey - hombre + mujer = ?`). Hay datasets estándar de analogías (Google Analogies, BATS).
*   **Evaluación Extrínseca:**
    *   Usar los embeddings como **características de entrada (features)** para un modelo de PLN más complejo que resuelve una tarea final (downstream task).
    *   Ejemplos: Clasificación de sentimientos, reconocimiento de entidades nombradas, clasificación de temas.
    *   Medir si usar estos embeddings mejora el rendimiento (precisión, F1-score, etc.) del sistema final comparado con no usarlos o usar otros embeddings. **Esta suele ser la evaluación más relevante en la práctica.**
*   **Detección de Sesgos:**
    *   **Analogías Específicas:** Crear analogías diseñadas para revelar sesgos sociales (género-profesión, raza-sentimiento, etc.). `programador - hombre + mujer = ?`.
    *   **Pruebas de Asociación Implícita (como WEAT - Word Embedding Association Test):** Miden matemáticamente qué tan asociados están ciertos grupos de palabras (ej: nombres masculinos vs femeninos) con ciertos atributos (ej: carreras científicas vs artísticas, adjetivos positivos vs negativos) dentro del espacio de embeddings.
    *   **Visualización:** A veces, proyectar grupos específicos (hombres/mujeres, profesiones) a 2D puede revelar agrupaciones o direcciones sesgadas.
    *   **Auditoría de Vecinos Cercanos:** Ver los `most_similar` para palabras sensibles.

**Una vez detectado el sesgo, ¿qué hacemos?** ¿Existen técnicas para mitigarlo ("debiasing")? ¿Es mejor buscar/crear corpus menos sesgados? ¿O simplemente ser transparentes sobre los sesgos del modelo?